In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualizations
import os
import glob

In [46]:
files = glob.glob("/home/dunbar/countminsketch/*.csv")
df = pd.read_csv(files[0])
for file in files[1:]:
    frame = pd.read_csv(file)
    df.append(frame,ignore_index=True)


In [47]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,5809910,1602943681873052386,NaN,grattol,5.24,595414620,4adb70bb-edbd-4981-b60f-a05bfd32683a
1,2020-01-01 00:00:09 UTC,view,5812943,1487580012121948301,NaN,kinetics,3.97,595414640,c8c5205d-be43-4f1d-aa56-4828b8151c8a
2,2020-01-01 00:00:19 UTC,view,5798924,1783999068867920626,NaN,zinger,3.97,595412617,46a5010f-bd69-4fbe-a00d-bb17aa7b46f3
3,2020-01-01 00:00:24 UTC,view,5793052,1487580005754995573,NaN,NaN,4.92,420652863,546f6af3-a517-4752-a98b-80c4c5860711
4,2020-01-01 00:00:25 UTC,view,5899926,2115334439910245200,NaN,NaN,3.92,484071203,cff70ddf-529e-4b0c-a4fc-f43a749c0acb


In [49]:
df.ndim


2

In [27]:
from random import *

N = 16
T = 100*1000

PRIMES = [121021, 121151, 150151, 151051, 151121, 180181, 180811, 181081, 2976221, 
          3021377, 6972593, 13466917, 20996011, 24036583, 25964951, 30402457]

def generate_hash(max_n, prime):
    p = prime
    n = max_n
    def hash_fn(val):
        return (val*p) % n
    return hash_fn

class CountMinSketch:
    def __init__(self):
        self.tables = [[0]*T for _ in range(0, N)]
        self.hashes = [generate_hash(T, PRIMES[i]) for i in range(0, N)]
        
    def add(self, val, count):
        hashes = self.hashes
        tables = self.tables
        for ix in range(0, N):
            h = hashes[ix](val)
            tables[ix][h] += count
    
    def estimate(self, value):
        results = []
        tables = self.tables
        hashes = self.hashes
        for ix in range(0, N):
            h = hashes[ix](value)
            c = tables[ix][h]
            results.append(c)
        return min(results)

In [17]:
seq1 = range(10000, 100000)
sketch1 = CountMinSketch()
for ix, x in enumerate(seq1):
    sketch1.add(x, ix + 1)

In [18]:
errors = []
for ix, x in enumerate(seq1):
    est = sketch1.estimate(x)
    err = float(est - ix - 1)/(ix+1)
    errors.append(err)